<a href="https://colab.research.google.com/github/Frederik-Roeckle/xwines_recom/blob/master/Recommender_Models/Graphs/GraphSAGE_XWines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

!pip install pyg-lib==0.4.0+pt26cu124 \
            torch-scatter==2.1.2+pt26cu124 \
            torch-sparse==0.6.18+pt26cu124 \
            torch-cluster==1.6.3+pt26cu124 \
            torch-spline-conv==1.2.2+pt26cu124 \
            -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

!pip install torch-geometric==2.5.0

#!pip install torch_geometric

# Optional dependencies:
#!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.5.0+cu124.html


Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidi

In [ ]:
!pip install torchmetrics

In [2]:
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv, to_hetero, GATv2Conv
from torch_geometric.loader import LinkNeighborLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from tqdm import tqdm
import pathlib as pl
from sentence_transformers import SentenceTransformer
import math
import copy
import datetime
from collections import defaultdict


In [3]:
# Get PyTorch version
pytorch_version = torch.__version__
print(f"PyTorch version: {pytorch_version}")

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

# Get CUDA version (if available)
if cuda_available:
    cuda_version = torch.version.cuda
    print(f"CUDA version: {cuda_version}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4


In [4]:
csv_wines = pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/XWines_Full_100K_wines.csv")
csv_training = pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/trainset.csv")

In [5]:
reviews = pd.read_csv(csv_training)

# for shorter training time prototyping reduce trainset size
# split = 0.3
# reviews = reviews.loc[reviews.index < len(reviews) * split]

all_wines = pd.read_csv(csv_wines)

wines = all_wines.loc[all_wines["WineID"].isin(reviews["WineID"])]

# 2. Encode categorical IDs
user_encoder = LabelEncoder()
wine_encoder = LabelEncoder()
reviews['UserID'] = user_encoder.fit_transform(reviews['UserID'])
wines['WineID'] = wine_encoder.fit_transform(wines['WineID'])
reviews['WineID'] = wine_encoder.transform(reviews['WineID'])

<ipython-input-5-98286d1bc806>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wines['WineID'] = wine_encoder.fit_transform(wines['WineID'])


In [6]:
def compute_grape_embeddings(wine_df):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  grape_sentences = wine_df['Grapes'].fillna("").tolist()
  grape_embeddings = model.encode(grape_sentences, show_progress_bar=True)

  # Convert to tensor
  return torch.tensor(grape_embeddings, dtype=torch.float)

grape_tensor = compute_grape_embeddings(wines)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3089 [00:00<?, ?it/s]

In [7]:
# User node features
global_rating_average = reviews["Rating"].mean()
user_rating_average = reviews.groupby("UserID")["Rating"].mean()
user_rating_average.name = "user_rating_average"

user_rating_count = reviews.groupby("UserID")["Rating"].count()
user_rating_count.name = "user_rating_count"

user_rating_std = reviews.groupby("UserID")["Rating"].std()
user_rating_std.name = "user_rating_std"

user_rating_bias = user_rating_average - global_rating_average
user_rating_bias.name = "user_rating_bias"

user_df = pd.concat([user_rating_average, user_rating_count, user_rating_std, user_rating_bias], axis=1)

# some users only have one rating, std then becomes nan leading to nan at loss computation
user_df = user_df.fillna(0)

# use scaler later for new user nodes
user_feature_scaler = StandardScaler()
user_df = user_feature_scaler.fit_transform(user_df[['user_rating_average', 'user_rating_count', 'user_rating_std', 'user_rating_bias']])
user_df

array([[ 8.68222297e-02,  1.77961218e+00,  2.13078121e-01,
         8.68222297e-02],
       [ 3.47530106e-01, -5.80055021e-04,  4.63838431e-01,
         3.47530106e-01],
       [ 3.26124617e-01,  8.55582788e-02, -8.80278512e-02,
         3.26124617e-01],
       ...,
       [ 4.83098202e-01, -3.45133390e-01,  9.56030573e-03,
         4.83098202e-01],
       [-3.15654362e-01,  6.02388281e-01, -9.42360368e-02,
        -3.15654362e-01],
       [-8.72582755e-01, -2.01569501e-01,  1.18972338e+00,
        -8.72582755e-01]])

In [8]:
# 3. Build Heterogeneous Graph
data = HeteroData()
data['user'].num_nodes = reviews['UserID'].nunique()
data['wine'].num_nodes = reviews['WineID'].nunique()

data['user', 'rates', 'wine'].edge_index = torch.tensor([
    reviews['UserID'].values,
    reviews['WineID'].values
], dtype=torch.long)

# Add ratings as edge weights
data['user', 'rates', 'wine'].edge_attr = torch.tensor(reviews['Rating'].values, dtype=torch.float).unsqueeze(1)

# Dummy features for users as no metadata is known
# Todo: Could encode average ratings
# data['user'].x = torch.ones((data['user'].num_nodes, 1))
data['user'].x = torch.tensor(user_df, dtype=torch.float)

def compute_standard_wine_features(wine_df):
  selected_columns = ['Type', 'Elaborate', 'Body', 'Acidity', 'Country']#, 'RegionID' , 'WineryID']
  wine_metadata = wine_df.set_index('WineID')[selected_columns]
  wine_metadata_encoded = pd.get_dummies(wine_metadata).astype(float)


# Use selected wine metadata as features
selected_columns = ['Type', 'Elaborate', 'Body', 'Acidity', 'Country']#, 'RegionID' , 'WineryID']
wine_metadata = wines.set_index('WineID')[selected_columns]
wine_metadata_encoded = pd.get_dummies(wine_metadata).astype(float)
training_wine_metadata_columns = wine_metadata_encoded.columns

# Align features to the wine node index order
aligned_wine_feats = wine_metadata_encoded.reindex(range(data['wine'].num_nodes)).fillna(0).values
# data['wine'].x = torch.tensor(aligned_wine_feats, dtype=torch.float)

# Concatenate features with grape features
structured_tensor = torch.tensor(aligned_wine_feats, dtype=torch.float)
combined_features = torch.cat([grape_tensor, structured_tensor], dim=1)
data['wine'].x = combined_features


<ipython-input-8-56dcd85b2af7>:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  data['user', 'rates', 'wine'].edge_index = torch.tensor([


In [ ]:
aligned_wine_feats.shape

(98825, 98)

In [ ]:
data

HeteroData(
  user={
    num_nodes=1056035,
    x=[1056035, 4],
  },
  wine={
    num_nodes=98825,
    x=[98825, 482],
  },
  (user, rates, wine)={
    edge_index=[2, 16917894],
    edge_attr=[16917894, 1],
  }
)

In [ ]:
data["user"]

{'num_nodes': 496265, 'x': tensor([[ 0.0402,  1.2863,  0.2783,  0.0402],
        [ 0.2898, -0.3607, -1.4947,  0.2898],
        [ 0.3797, -0.0113,  0.4846,  0.3797],
        ...,
        [ 0.2898, -0.4605, -1.4947,  0.2898],
        [-0.1596,  1.2863,  0.0732, -0.1596],
        [-0.6090, -0.0612,  1.1030, -0.6090]])}

In [9]:
# 5. Define GraphSAGE Model
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.relu = nn.ReLU()
        self.conv2 = SAGEConv((-1, -1), hidden_channels)

    def forward(self, x_dict, edge_index_dict):
        x = self.conv1(x_dict, edge_index_dict)
        x = self.relu(x)
        x = self.conv2(x, edge_index_dict)
        return x

In [10]:
class MLPPredictor(nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        self.fc1 = nn.Linear(embedding_dim * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, user_emb, wine_emb):
        x = torch.cat([user_emb, wine_emb], dim=-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x).squeeze(-1)

In [11]:
# 7. Prepare train/val split (temporal or random)
edge_index = data['user', 'rates', 'wine'].edge_index
num_edges = edge_index.size(1)
split_idx = int(0.9 * num_edges)

train_edge_index = edge_index[:, :split_idx]
val_edge_index = edge_index[:, split_idx:]

data['user', 'rates', 'wine'].edge_index = train_edge_index

# Opt
data['wine', 'rev_rates', 'user'].edge_index = data['user', 'rates', 'wine'].edge_index[[1, 0]]

data['user', 'rates', 'wine'].train_edge_index = train_edge_index

data['user', 'rates', 'wine'].val_edge_index = val_edge_index

In [12]:
# 8. Create loader for training
train_loader = LinkNeighborLoader(
    data,
    num_neighbors=[25, 10],
    batch_size=8192,
    edge_label_index=("user", "rates", "wine"),
    edge_label=torch.tensor(reviews['Rating'].values[:split_idx], dtype=torch.float),
    shuffle=True
)

val_loader = LinkNeighborLoader(
    data=data,
    num_neighbors=[25, 10],
    batch_size=8192,
    edge_label_index=(('user', 'rates', 'wine'), val_edge_index),
    edge_label=torch.tensor(reviews['Rating'].values[split_idx:], dtype=torch.float),
    shuffle=False
)

In [29]:
encoder = GNNEncoder(hidden_channels=64)
model = to_hetero(encoder, data.metadata(), aggr='sum').to(device)#
predictor = MLPPredictor(64).to(device)

optimizer = torch.optim.Adam(list(model.parameters()) + list(predictor.parameters()), lr=0.01)

model.train()
predictor.train()

for epoch in range(10):
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = batch.to(device)
        optimizer.zero_grad()

        # Forward pass
        z_dict = model(batch.x_dict, batch.edge_index_dict)

        # Get the user and wine nodes involved in the prediction task
        user_id = batch["user", "rates", "wine"].edge_label_index[0]
        wine_id = batch["user", "rates", "wine"].edge_label_index[1]

        # Fetch their embeddings
        user_emb = z_dict['user'][user_id]
        wine_emb = z_dict['wine'][wine_id]

        # Predict ratings with MLP
        pred = predictor(user_emb, wine_emb)

        # Compute loss
        target = batch["user", "rates", "wine"].edge_label.squeeze().float()
        loss = F.mse_loss(pred, target)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch}, Loss: {total_loss:.4f}")


100%|██████████| 1859/1859 [11:55<00:00,  2.60it/s]


Epoch 0, Loss: 764.5010


100%|██████████| 1859/1859 [11:55<00:00,  2.60it/s]


Epoch 1, Loss: 646.2703


100%|██████████| 1859/1859 [11:56<00:00,  2.59it/s]


...

Epoch 7, Loss: 625.3758


100%|██████████| 1859/1859 [11:58<00:00,  2.59it/s]


Epoch 8, Loss: 623.9639


100%|██████████| 1859/1859 [11:58<00:00,  2.59it/s]

Epoch 9, Loss: 623.0961


In [ ]:
data

HeteroData(
  user={
    num_nodes=1056035,
    x=[1056035, 4],
  },
  wine={
    num_nodes=98825,
    x=[98825, 482],
  },
  (user, rates, wine)={
    edge_index=[2, 15226104],
    edge_attr=[16917894, 1],
    train_edge_index=[2, 15226104],
    val_edge_index=[2, 1691790],
  },
  (wine, rev_rates, user)={ edge_index=[2, 15226104] }
)

In [13]:
# Load previously stored models
data = torch.load(pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/full_3_data.pth"), weights_only=False)

encoder = GNNEncoder(hidden_channels=64)
model = to_hetero(encoder, data.metadata(), aggr='sum').to(device)#
model.load_state_dict(torch.load(pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/full_3_model.pth"), weights_only=True))

predictor = MLPPredictor(64).to(device)
predictor.load_state_dict(torch.load(pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/full_3_predictor.pth"), weights_only=True))

<All keys matched successfully>

In [30]:
torch.save(model.state_dict(), pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject") / "full_10_model.pth")
torch.save(predictor.state_dict(), pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject") / "full_10_predictor.pth")
torch.save(data, pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject") / "full_10_data.pth")

In [14]:
original_data = copy.deepcopy(data)

In [31]:
model.eval()
predictor.eval()

total_loss = 0
all_preds = []
all_targets = []

with torch.no_grad():
    for batch in tqdm(val_loader):  # LinkNeighborLoader for val edges
        batch = batch.to(device)
        z_dict = model(batch.x_dict, batch.edge_index_dict)

        user_id = batch["user", "rates", "wine"].edge_label_index[0]
        wine_id = batch["user", "rates", "wine"].edge_label_index[1]

        user_emb = z_dict['user'][user_id]
        wine_emb = z_dict['wine'][wine_id]

        preds = predictor(user_emb, wine_emb)
        targets = batch["user", "rates", "wine"].edge_label.squeeze().float()

        loss = F.mse_loss(preds, targets)
        total_loss += loss.item() * preds.size(0)

        all_preds.append(preds.cpu())
        all_targets.append(targets.cpu())

avg_loss = total_loss / len(val_loader.dataset)
rmse = torch.sqrt(torch.tensor(avg_loss))
print(f"Validation RMSE: {rmse.item():.4f}")


100%|██████████| 207/207 [01:08<00:00,  3.02it/s]

Validation RMSE: 0.5977


## Evaluation on Test segments

In [32]:
csv_testset_warm_user_warm_item = pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/testset_warm_user_warm_item.csv")
csv_testset_warm_user_cold_item = pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/testset_warm_user_cold_item.csv")
csv_testset_cold_user_warm_item = pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/testset_cold_user_warm_item.csv")
csv_testset_cold_user_cold_item = pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/testset_cold_user_cold_item.csv")

testset_warm_user_warm_item = pd.read_csv(csv_testset_warm_user_warm_item)
testset_warm_user_cold_item = pd.read_csv(csv_testset_warm_user_cold_item)
testset_cold_user_warm_item = pd.read_csv(csv_testset_cold_user_warm_item)
testset_cold_user_cold_item = pd.read_csv(csv_testset_cold_user_cold_item)


test_segments = [testset_warm_user_warm_item, testset_warm_user_cold_item, testset_cold_user_warm_item, testset_cold_user_cold_item]

In [33]:
def process_new_wines(new_wine_ids, wine_df, wine_encoder, training_metadata_columns):
    # Filter new wines
    new_wine_df = wine_df[wine_df['WineID'].isin(new_wine_ids)].copy()

    # Map using existing encoder
    next_wine_index = len(original_wine_id_to_index)

    for wid in new_wine_ids:
        if wid not in wine_id_to_index:
            wine_id_to_index[wid] = next_wine_index
            next_wine_index += 1

    grape_tensor = compute_grape_embeddings(new_wine_df)

    # Step 2: Encode metadata using one-hot encoding aligned to training columns
    selected_columns = ['Type', 'Elaborate', 'Body', 'Acidity', 'Country']
    metadata = new_wine_df.set_index('WineID')[selected_columns]
    metadata_encoded = pd.get_dummies(metadata).astype(float)

    # Align with training columns (fill missing with 0)
    metadata_encoded = metadata_encoded.reindex(columns=training_metadata_columns, fill_value=0.0)
    structured_tensor = torch.tensor(metadata_encoded.values, dtype=torch.float)

    # Step 3: Concatenate grape and structured features
    combined_features = torch.cat([grape_tensor, structured_tensor], dim=1)

    return combined_features


In [34]:
# Alignment bug when it comes to the Loader Function -> Wine Encoding probably

# for handling unknown users and items later on
original_user_id_to_index = {id_: idx for idx, id_ in enumerate(user_encoder.classes_)}
original_wine_id_to_index = {id_: idx for idx, id_ in enumerate(wine_encoder.classes_)}

for segment_id, segment_df in enumerate(test_segments):
    print(f"Evaluating Segment {segment_id}...")

    # Step 1: Reset graph and ID mappings
    data = copy.deepcopy(original_data)
    user_id_to_index = copy.deepcopy(original_user_id_to_index)
    wine_id_to_index = copy.deepcopy(original_wine_id_to_index)

    # Step 2: Detect new user and wine IDs
    new_user_ids = segment_df['UserID'][~segment_df['UserID'].isin(user_id_to_index)].unique()
    print(f"new users in test segment: {len(new_user_ids)}")

    new_wine_ids = segment_df['WineID'][~segment_df['WineID'].isin(wine_id_to_index)].unique()
    print(f"new wines in test segment: {len(new_wine_ids)}")
    new_wine_combined_features = process_new_wines(new_wine_ids, all_wines, wine_encoder, training_wine_metadata_columns)
    print(f"shape of new wine features {new_wine_combined_features.shape}")

    # Step 3: Assign new indices (continue from current num_nodes)
    current_user_count = data['user'].num_nodes
    current_wine_count = data['wine'].num_nodes

    for uid in new_user_ids:
        user_id_to_index[uid] = current_user_count
        current_user_count += 1

    for wid in new_wine_ids:
        wine_id_to_index[wid] = current_wine_count
        current_wine_count += 1

    # Step 4: Extend feature matrices for new nodes
    num_new_users = current_user_count - data['user'].num_nodes
    num_new_wines = current_wine_count - data['wine'].num_nodes

    user_feat_dim = data['user'].x.size(1)
    wine_feat_dim = data['wine'].x.size(1)

    # no metadata for users therefore zeros
    new_user_feats = torch.zeros((num_new_users, user_feat_dim), dtype=torch.float, device=data['user'].x.device)
    new_wine_feats = torch.tensor(new_wine_combined_features, dtype=torch.float, device=data['wine'].x.device)
    print(f"new_wine_feats: {new_wine_feats}")
    if new_wine_feats.size(0) > 0:
      data['wine'].x = torch.cat([data['wine'].x, new_wine_feats], dim=0)

    if new_user_feats.size(0) > 0:
      data['user'].x = torch.cat([data['user'].x, new_user_feats], dim=0)

    data['user'].num_nodes = current_user_count
    data['wine'].num_nodes = current_wine_count


    # Construct test edges
    user_indices = torch.tensor([user_id_to_index[uid] for uid in segment_df['UserID']], dtype=torch.long)
    wine_indices = torch.tensor([wine_id_to_index[wid] for wid in segment_df['WineID']], dtype=torch.long)
    ratings = torch.tensor(segment_df['Rating'].values, dtype=torch.float)

    test_edge_index = torch.stack([user_indices, wine_indices], dim=0)

    test_loader = LinkNeighborLoader(
        data=data,
        num_neighbors=[25, 10],
        edge_label_index=(('user', 'rates', 'wine'), test_edge_index),
        edge_label=torch.tensor(ratings, dtype=torch.float),
        batch_size=1024,
        shuffle=False
    )

    model.eval()
    predictor.eval()
    preds = []
    trues = []


    precisions = []
    avg_ndcg = []

    with torch.no_grad():
        for batch in tqdm(test_loader):
            batch = batch.to(device)
            z_dict = model(batch.x_dict, batch.edge_index_dict)

            user_id = batch["user", "rates", "wine"].edge_label_index[0]
            wine_id = batch["user", "rates", "wine"].edge_label_index[1]

            user_emb = z_dict['user'][user_id]
            wine_emb = z_dict['wine'][wine_id]

            pred = predictor(user_emb, wine_emb)
            preds.append(pred.cpu())
            trues.append(batch["user", "rates", "wine"].edge_label.cpu())

    preds = torch.cat(preds)
    trues = torch.cat(trues)

    # save preds and trues
    torch.save(preds, pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject") / f"preds_segment_{segment_id}_at_10_epochs.pt")
    torch.save(trues, pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject") / f"trues_segment_{segment_id}_at_10_epochs.pt")

    rmse = torch.sqrt(F.mse_loss(preds, trues))
    print(f"Segment {segment_id} RMSE: {rmse:.4f}")

Evaluating Segment 0...
new users in test segment: 5
new wines in test segment: 0


Batches: 0it [00:00, ?it/s]

shape of new wine features torch.Size([0, 98])
new_wine_feats: tensor([], size=(0, 98))


<ipython-input-34-583445ce2082>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_wine_feats = torch.tensor(new_wine_combined_features, dtype=torch.float, device=data['wine'].x.device)
<ipython-input-34-583445ce2082>:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_label=torch.tensor(ratings, dtype=torch.float),
100%|██████████| 1990/1990 [04:55<00:00,  6.74it/s]


Segment 0 RMSE: 0.6504
Evaluating Segment 1...
new users in test segment: 1
new wines in test segment: 1321


...

Segment 2 RMSE: 1.0213
Evaluating Segment 3...
new users in test segment: 44
new wines in test segment: 1515


Batches:   0%|          | 0/48 [00:00<?, ?it/s]

shape of new wine features torch.Size([1515, 482])
new_wine_feats: tensor([[ 0.0246, -0.0006, -0.1046,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0056, -0.0242, -0.0746,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0056, -0.0242, -0.0746,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.0047,  0.1094, -0.0786,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0410,  0.0567, -0.1049,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0434, -0.0570, -0.0825,  ...,  0.0000,  0.0000,  0.0000]])


<ipython-input-34-583445ce2082>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_wine_feats = torch.tensor(new_wine_combined_features, dtype=torch.float, device=data['wine'].x.device)
<ipython-input-34-583445ce2082>:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_label=torch.tensor(ratings, dtype=torch.float),
100%|██████████| 17/17 [00:01<00:00, 11.67it/s]

Segment 3 RMSE: 0.9680


In [ ]:
print(user_indices)
print(wine_indices)
print(ratings)
print(test_edge_index)

tensor([ 840559,   61576,  305883,  ...,  967035, 1000261, 1018508])
tensor([ 98825,  98826,  98827,  ...,  99997, 100322, 100093])
tensor([5.0000, 5.0000, 3.5000,  ..., 5.0000, 4.0000, 4.0000])
tensor([[ 840559,   61576,  305883,  ...,  967035, 1000261, 1018508],
        [  98825,   98826,   98827,  ...,   99997,  100322,  100093]])


In [ ]:
for user_id, uid in user_id_to_index.items():
  if uid == 1018508:
    print(f"real user_id: {user_id}, user_id in graph strut {uid}")
for wine_id, wid in wine_id_to_index.items():
  if wid == 100093:
    print(wine_id, wid)

print(user_id_to_index[1846409])
print(wine_id_to_index[105703])

real user_id: 2025644, user_id in graph strut 1018508
165970 100093
840559
98825


In [ ]:
testset_cold_user_cold_item.loc[testset_cold_user_cold_item['UserID'] == 2025644]

,RatingID,UserID,WineID,Vintage,Rating,Date
16503,15236817,2025644,165970,2017,4.0,2021-01-03 13:33:05


## Top-k metric evaluation of GraphSAGE

Not possible to compute due to scale of dataset and nature of GraphSAGE

Evaluating model performance on standard recommender model metrices.

- NDCG
- Accuracy@k
- Precision@k
- Recall@k
- Hitrate@k



In [35]:
def evaluate_segment_with_ranking(model, predictor, original_data, test_segments, all_wines,
                                   wine_encoder, user_encoder, training_wine_metadata_columns,
                                   process_new_wines, device='cuda', k=10):

    original_user_id_to_index = {id_: idx for idx, id_ in enumerate(user_encoder.classes_)}
    original_wine_id_to_index = {id_: idx for idx, id_ in enumerate(wine_encoder.classes_)}

    for segment_id, segment_df in enumerate(test_segments):
        print(f"Evaluating Segment {segment_id}...")

        data = copy.deepcopy(original_data)
        user_id_to_index = copy.deepcopy(original_user_id_to_index)
        wine_id_to_index = copy.deepcopy(original_wine_id_to_index)

        new_user_ids = segment_df['UserID'][~segment_df['UserID'].isin(user_id_to_index)].unique()
        new_wine_ids = segment_df['WineID'][~segment_df['WineID'].isin(wine_id_to_index)].unique()

        new_wine_combined_features = process_new_wines(new_wine_ids, all_wines, wine_encoder, training_wine_metadata_columns)

        current_user_count = data['user'].num_nodes
        current_wine_count = data['wine'].num_nodes

        for uid in new_user_ids:
            user_id_to_index[uid] = current_user_count
            current_user_count += 1

        for wid in new_wine_ids:
            wine_id_to_index[wid] = current_wine_count
            current_wine_count += 1

        num_new_users = current_user_count - data['user'].num_nodes
        num_new_wines = current_wine_count - data['wine'].num_nodes

        user_feat_dim = data['user'].x.size(1)
        wine_feat_dim = data['wine'].x.size(1)

        new_user_feats = torch.zeros((num_new_users, user_feat_dim), dtype=torch.float, device=device)
        new_wine_feats = torch.tensor(new_wine_combined_features, dtype=torch.float, device=device)

        if new_wine_feats.size(0) > 0:
            data['wine'].x = torch.cat([data['wine'].x, new_wine_feats], dim=0)
        if new_user_feats.size(0) > 0:
            data['user'].x = torch.cat([data['user'].x, new_user_feats], dim=0)

        data['user'].num_nodes = current_user_count
        data['wine'].num_nodes = current_wine_count

        ground_truth = defaultdict(set)
        for _, row in segment_df.iterrows():
            uid = row['UserID']
            wid = row['WineID']
            if uid in user_id_to_index and wid in wine_id_to_index:
                ground_truth[user_id_to_index[uid]].add(wine_id_to_index[wid])

        all_users = list(ground_truth.keys())
        all_wine_ids = torch.arange(data['wine'].x.size(0), device=device)

        model.eval()
        predictor.eval()

        precisions = []
        ndcgs = []

        with torch.no_grad():
            z_dict = model(data.x_dict, data.edge_index_dict)
            wine_emb = z_dict['wine'][all_wine_ids]

            for user_idx in tqdm(all_users, desc=f"Ranking Evaluation Segment {segment_id}"):
                print(f"Calculating for user: {user_idx}")
                user_emb = z_dict['user'][user_idx].unsqueeze(0).repeat(len(all_wine_ids), 1)
                scores = predictor(user_emb, wine_emb).squeeze()
                topk_scores, topk_indices = torch.topk(scores, k)
                topk_items = topk_indices.tolist()

                relevance = torch.tensor([1 if item in ground_truth[user_idx] else 0 for item in topk_items], device=device)

                precisions.append(retrieval_precision(relevance.unsqueeze(0), torch.ones_like(relevance).unsqueeze(0), k=k).item())
                ndcgs.append(retrieval_normalized_dcg(relevance.unsqueeze(0), torch.ones_like(relevance).unsqueeze(0), k=k).item())

        avg_precision = sum(precisions) / len(precisions)
        avg_ndcg = sum(ndcgs) / len(ndcgs)

        print(f"Segment {segment_id} Precision@{k}: {avg_precision:.4f}")
        print(f"Segment {segment_id} NDCG@{k}: {avg_ndcg:.4f}")


In [36]:
original_user_id_to_index = {id_: idx for idx, id_ in enumerate(user_encoder.classes_)}
original_wine_id_to_index = {id_: idx for idx, id_ in enumerate(wine_encoder.classes_)}

k = 15
data.to(device)
model.to(device)
predictor.to(device)

evaluate_segment_with_ranking(model, predictor, data, test_segments, wines, wine_encoder, user_encoder, training_wine_metadata_columns, process_new_wines, device, k=15)

Evaluating Segment 0...


Batches: 0it [00:00, ?it/s]

<ipython-input-35-17db5ff8f49f>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_wine_feats = torch.tensor(new_wine_combined_features, dtype=torch.float, device=device)


OutOfMemoryError: CUDA out of memory. Tried to allocate 27.34 GiB. GPU 0 has a total capacity of 22.16 GiB of which 15.10 GiB is free. Process 5140 has 7.06 GiB memory in use. Of the allocated memory 4.02 GiB is allocated by PyTorch, and 2.81 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)